In [1]:
import pandas as pd
import re
import numpy as np

In [23]:
import duckdb

def sql(query):
    con = duckdb.connect()
    res = con.execute(query).df()
    con.close()
    return res

df_full = sql("SELECT * FROM read_csv('../data/df_clean.csv')")
df_full

,title,brand,price,rating,votes,sales_last_month,image_url,url,prime,category,rank,scraped_at
0,Garmin – Forerunner 55 - Montre GPS multi-acti...,Garmin,138.99,4.6,6532.0,1.0,https://m.media-amazon.com/images/I/61eTqEILa9...,https://www.amazon.fr/Garmin-Forerunner-multi-...,False,Objets connectés,1,2025-06-24 14:20:21.796320
1,Garmin Forerunner 255 - Montre GPS Multisports...,Garmin,203.99,4.5,1690.0,700.0,https://m.media-amazon.com/images/I/71tjy7Umf0...,https://www.amazon.fr/Garmin-GPS-Forerunner-25...,False,Objets connectés,2,2025-06-24 14:20:21.799403
2,"Garmin Forerunner 255, Music - Montre GPS Mult...",Garmin,244.00,4.5,616.0,700.0,https://m.media-amazon.com/images/I/51BxNNStky...,https://www.amazon.fr/Garmin-Forerunner-255-Mu...,False,Objets connectés,3,2025-06-24 14:20:21.802907
3,"Insta360 X5 - caméra d'action 360° 8K étanche,...",Insta360,589.99,4.5,559.0,300.0,https://m.media-amazon.com/images/I/71YjsbkkO9...,https://www.amazon.fr/Insta360-Invisible-rempl...,False,Objets connectés,4,2025-06-24 14:20:21.805683
4,Garmin Forerunner 255 – GPS-Laufuhr mit indivi...,Garmin,203.99,4.6,662.0,400.0,https://m.media-amazon.com/images/I/51oM3U58x2...,https://www.amazon.fr/Garmin-GPS-Forerunner-25...,False,Objets connectés,5,2025-06-24 14:20:21.808207
...,...,...,...,...,...,...,...,...,...,...,...,...
4906,Traqueur GPS pour enfants - Clés de chien cach...,Traqueur,30.99,1.0,1.0,NaN,https://m.media-amazon.com/images/I/71ll8TdY-4...,https://www.amazon.fr/Traqueur-GPS-pour-enfant...,True,Objets connectés,47941,2025-06-24 16:47:23.080776
4907,"Lot de 2 traqueurs GPS pour enfants, clés de c...",Lot,25.99,4.7,7.0,NaN,https://m.media-amazon.com/images/I/71Axw1YHuY...,https://www.amazon.fr/traqueurs-anti-retrait-f...,True,Objets connectés,47964,2025-06-24 16:47:27.311373
4908,Traqueur GPS pour enfants - Clés de chien cach...,Traqueur,30.99,1.0,1.0,NaN,https://m.media-amazon.com/images/I/71ll8TdY-4...,https://www.amazon.fr/Traqueur-GPS-pour-enfant...,True,Objets connectés,47965,2025-06-24 16:47:27.312839
4909,"Lot de 2 traqueurs GPS pour enfants, clés de c...",Lot,25.99,4.7,7.0,NaN,https://m.media-amazon.com/images/I/71Axw1YHuY...,https://www.amazon.fr/traqueurs-anti-retrait-f...,True,Objets connectés,47988,2025-06-24 16:47:31.542076


In [36]:
#--------------------scrapping des produits a partir de l'url du scrapping initial--------------------

import requests
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import random

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.1 Safari/605.1.15",
    "Accept-Language": "fr-FR,fr;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
}

prod = []

for u in tqdm(df_full['url']):
    time.sleep(random.uniform(1, 2.5))
    try:
        r = requests.get(u, headers=headers, timeout=10)

        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')

            # Catégorie
            card = soup.find('ul', class_='a-unordered-list a-horizontal a-size-small')
            cat = card.get_text(" > ", strip=True) if card else None

            # Titre
            title_tag = soup.find('span', class_="a-size-large product-title-word-break")
            title = title_tag.get_text(strip=True) if title_tag else None

            # Classement / Caractéristiques
            top_list = []
            top_card = soup.find('ul', class_="a-unordered-list a-nostyle a-vertical")
            if top_card:
                top_soup = top_card.find_all('span', class_="a-list-item")
                top_list = [t.get_text(" ",strip=True) for t in top_soup]

            # Date de mise en ligne
            date_pub = None
            table_soup = soup.find('table', id='productDetails_detailBullets_sections1')
            if table_soup:
                rows = table_soup.find_all('tr')
                for row in rows:
                    th = row.find('th')
                    td = row.find('td')
                    if th and "date de mise en ligne" in th.get_text(strip=True).lower():
                        date_pub = td.get_text(strip=True) if td else None
                        break

            # Stockage
            prod.append({
                "titre": title,
                "url": u,
                "categorie": cat,
                "classement": top_list,
                "date_mise_en_ligne": date_pub
            })

        else:
            print("❌", r.status_code, u)

    except Exception as e:
        print(f"💥 Erreur pour {u} : {e}")

# Convertir en DataFrame
df_complément = pd.DataFrame(prod)



  0%|          | 3/4560 [00:14<6:16:46,  4.96s/it]


KeyboardInterrupt: 

,url,titre,categorie,classement,date_mise_en_ligne
0,https://www.amazon.fr/PowerLocus-Bluetooth-%C3...,"PowerLocus Casque Bluetooth Enfant, P2 Casque ...","High-Tech > › > Casques, écouteurs et accessoi...",[9 370 en High-Tech ( Voir les 100 premiers en...,1 juillet 2020
1,https://www.amazon.fr/Tensiom%C3%A8tre-OMRON-C...,OMRON RS3 Intelli IT tensiomètre automatique a...,Hygiène et Santé > › > Matériel et fournitures...,[6 008 en Hygiène et Santé ( Voir les 100 prem...,25 janvier 2019
2,https://www.amazon.fr/Xmenha-%C3%89couteurs-Bl...,Xmenha Écouteurs Bluetooth Écran Touché Contrô...,"High-Tech > › > Casques, écouteurs et accessoi...",[537 836 en High-Tech ( Voir les 100 premiers ...,12 avril 2024
3,https://www.amazon.fr/%C3%89couteurs-EDX-Pro-a...,Yinyoo KZ EDX Pro Écouteurs filaires avec Micr...,"High-Tech > › > Casques, écouteurs et accessoi...",[313 844 en High-Tech ( Voir les 100 premiers ...,18 novembre 2021
4,https://www.amazon.fr/Interrupteur-Commutateur...,Disjoncteur WIFI intelligent Interrupteur Conn...,Informatique > › > Réseaux > › > Switches et hubs,[],None
...,...,...,...,...,...
4553,https://www.amazon.fr/POFET-batterie-suppl%C3%...,None,None,[],None
4554,https://www.amazon.fr/%C3%89couteurs-%C3%A9cou...,None,None,[],None
4555,https://www.amazon.fr/hisonic-Casque-pour-enfa...,None,None,[],None
4556,https://www.amazon.fr/Diyeeni-Objectif-Corpore...,"Diyeeni Collier de Caméra pour Chat, Caméra po...","Animalerie > › > Chats > › > Colliers, harnais...",[],None


In [211]:
df_complément = pd.read_parquet("../produits_amazon_scrapes.parquet")
df_complément

,url,titre,categorie,classement,date_mise_en_ligne
0,https://www.amazon.fr/PowerLocus-Bluetooth-%C3...,"PowerLocus Casque Bluetooth Enfant, P2 Casque ...","High-Tech > › > Casques, écouteurs et accessoi...",[9 370 en High-Tech ( Voir les 100 premiers en...,1 juillet 2020
1,https://www.amazon.fr/Tensiom%C3%A8tre-OMRON-C...,OMRON RS3 Intelli IT tensiomètre automatique a...,Hygiène et Santé > › > Matériel et fournitures...,[6 008 en Hygiène et Santé ( Voir les 100 prem...,25 janvier 2019
2,https://www.amazon.fr/Xmenha-%C3%89couteurs-Bl...,Xmenha Écouteurs Bluetooth Écran Touché Contrô...,"High-Tech > › > Casques, écouteurs et accessoi...",[537 836 en High-Tech ( Voir les 100 premiers ...,12 avril 2024
3,https://www.amazon.fr/%C3%89couteurs-EDX-Pro-a...,Yinyoo KZ EDX Pro Écouteurs filaires avec Micr...,"High-Tech > › > Casques, écouteurs et accessoi...",[313 844 en High-Tech ( Voir les 100 premiers ...,18 novembre 2021
4,https://www.amazon.fr/Interrupteur-Commutateur...,Disjoncteur WIFI intelligent Interrupteur Conn...,Informatique > › > Réseaux > › > Switches et hubs,[],None
...,...,...,...,...,...
4553,https://www.amazon.fr/POFET-batterie-suppl%C3%...,None,None,[],None
4554,https://www.amazon.fr/%C3%89couteurs-%C3%A9cou...,None,None,[],None
4555,https://www.amazon.fr/hisonic-Casque-pour-enfa...,None,None,[],None
4556,https://www.amazon.fr/Diyeeni-Objectif-Corpore...,"Diyeeni Collier de Caméra pour Chat, Caméra po...","Animalerie > › > Chats > › > Colliers, harnais...",[],None


In [212]:
#--------------------Nettoyage des données--------------------

def clean_class(liste_class):
    class_clean = []
    for l in liste_class:
        # Retirer les parenthèses et leur contenu
        l = re.sub(r"\(.*?\)", "", l).strip()

        # Remplacer les espaces insécables et normales par rien dans les nombres
        l = re.sub(r"(\d)[\s ](\d)", r"\1\2", l)  # supprime espace entre chiffres

        # Appliquer l'extraction
        matches = re.findall(r"(\d+)\s+en\s+([a-zA-ZÀ-ÿ\s\-']+)", l)
        for nb, cat in matches:
            class_clean.append((int(nb), cat.strip()))
    return class_clean


def drop_list_dupl(ls):
    cleaned_list = []
    for l in ls : 
        if l not in cleaned_list : 
            cleaned_list.append(l)
        else : 
            continue
    return cleaned_list

def extract_cat():
    list_cat = []
    for li in df_complément['classement'] :
        for cat in li : 
            #print(cat[1].replace(" ", "_"))
            list_cat.append(cat[1].replace(" ", "_"))
    unique_list_unique = drop_list_dupl(list_cat)
    return unique_list_unique

#clean initial catégories
df_complément['categorie'] = df_complément['categorie'].astype(str).str.replace(">", "", regex=False).str.replace("›", ".",regex=False).str.split(".").apply(lambda lst : [l.strip() for l in lst])

#clean de la colonne classement
df_complément['classement'] = df_complément['classement'].apply(clean_class)

#extraction des catégories
unique_list_unique = extract_cat()

print(unique_list_unique)

#création des colonnes catégories
df_complément[unique_list_unique] = np.nan


#remplissage des colonnes classement avec les valeurs correspondantes
for idx, classement in enumerate(df_complément['classement']):
    if isinstance(classement, list):  # pour éviter les NaN
        for j in classement:
            # j = (rang, categorie)
            categorie = j[1]
            rang = j[0]
            df_complément.at[idx, categorie] = rang
df_complément


['High-Tech', 'Casques_et_écouteurs', 'Hygiène_et_Santé', 'Tensiomètres', 'Écouteurs_à_conduction_osseuse', 'Mode', 'Montres_de_poche_et_goussets_pour_homme', 'Enceintes_Bluetooth_portables', 'Écouteurs_intra-auriculaires', 'Casques_de_réalité_virtuelle__pour_téléphone_portable', 'Caméras_embarquées', 'Jeux_vidéo', 'Accessoires_de_réalité_virtuelle_autonome', "Localisateurs_d'objets", 'Sports_et_Loisirs', "Trackers_d'activité", 'Caméras_de_corps', 'GPS_de_running', 'Enceintes_PC', "Enceintes_portables_et_stations_d'accueil", 'Montres_connectées', 'Batteries_pour_lecteurs_MP', 'Climate_Pledge_Friendly', 'Claviers', 'Chargeurs_secteur_pour_téléphones_portables', 'Housses_de_protection', 'Pèse-personne_numériques', 'Montres_connectées_tendance_pour_homme', 'Tapis_de_souris', 'Bricolage', 'Sirènes_de_sécurité', 'Enceintes', 'Instruments_de_musique_et_Sono', 'Casques_pour_home_studio', 'Étuis_et_housses_pour_casques_et_écouteurs', 'Lecteurs_de_carte_mémoire_externes', 'Bateaux_et_navigation

/var/folders/w7/mwhn41wd4133mpd3qmk0b4lh0000gn/T/ipykernel_50110/848029247.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_complément[unique_list_unique] = np.nan
/var/folders/w7/mwhn41wd4133mpd3qmk0b4lh0000gn/T/ipykernel_50110/848029247.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_complément[unique_list_unique] = np.nan
/var/folders/w7/mwhn41wd4133mpd3qmk0b4lh0000gn/T/ipykernel_50110/848029247.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

,url,titre,categorie,classement,date_mise_en_ligne,High-Tech,Casques_et_écouteurs,Hygiène_et_Santé,Tensiomètres,Écouteurs_à_conduction_osseuse,...,Housses et étuis pour GPS,Auto et Moto,Bracelets de montres pour femme,Univers Hi-Fi,Cassettes audio,Microphones pour radios,Cartes graphiques,Bracelets énergétiques,Accessoires GPS,Réhausseurs enceintes
0,https://www.amazon.fr/PowerLocus-Bluetooth-%C3...,"PowerLocus Casque Bluetooth Enfant, P2 Casque ...","[High-Tech, Casques, écouteurs et accessoires,...","[(9370, High-Tech), (709, Casques et écouteurs)]",1 juillet 2020,9370.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.amazon.fr/Tensiom%C3%A8tre-OMRON-C...,OMRON RS3 Intelli IT tensiomètre automatique a...,"[Hygiène et Santé, Matériel et fournitures méd...","[(6008, Hygiène et Santé), (33, Tensiomètres)]",25 janvier 2019,NaN,NaN,NaN,33.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.amazon.fr/Xmenha-%C3%89couteurs-Bl...,Xmenha Écouteurs Bluetooth Écran Touché Contrô...,"[High-Tech, Casques, écouteurs et accessoires,...","[(537836, High-Tech), (2869, Écouteurs à condu...",12 avril 2024,537836.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.amazon.fr/%C3%89couteurs-EDX-Pro-a...,Yinyoo KZ EDX Pro Écouteurs filaires avec Micr...,"[High-Tech, Casques, écouteurs et accessoires,...","[(313844, High-Tech), (6886, Casques et écoute...",18 novembre 2021,313844.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.amazon.fr/Interrupteur-Commutateur...,Disjoncteur WIFI intelligent Interrupteur Conn...,"[Informatique, Réseaux, Switches et hubs]",[],None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4553,https://www.amazon.fr/POFET-batterie-suppl%C3%...,None,[None],[],None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4554,https://www.amazon.fr/%C3%89couteurs-%C3%A9cou...,None,[None],[],None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4555,https://www.amazon.fr/hisonic-Casque-pour-enfa...,None,[None],[],None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4556,https://www.amazon.fr/Diyeeni-Objectif-Corpore...,"Diyeeni Collier de Caméra pour Chat, Caméra po...","[Animalerie, Chats, Colliers, harnais et laiss...",[],None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [209]:
df_complément.at[4, 'url']

'https://www.amazon.fr/Interrupteur-Commutateur-T%C3%A9l%C3%A9phone-T%C3%A9l%C3%A9commande-Intelligente/dp/B0C4YTSZ8J/ref=sr_1_1091?dib=eyJ2IjoiMSJ9.E8lYCkBvJQSSHjGweAC4THlDGjKPFE3kp4a-6ydtTY-bJmzAJxZLKdEEIYWZQmLNu8-2XYXVIg0_xiaTkf0YJ7Wc2TWaKMy0ogF3Wvlidw4-6yJLxiBk59J662JD-nywlu9DvLwYX1Sb0mRMVUQKFG78AtGxIl5hWrlCQy2eNpMZR7BigcFT1c_u_f2e8jqhjpU0O6pjqKqv7ebCKqjrSYlGbmIdY3BHkMQTx9-NUqCyKFewQBBBvAFWOsGu2_MXWpxxrPtrQmoarNEvnLtaUYsa6OQdT7JMahKWDkJQqz0.Z0uQCsVRswOrA1uq59ZADZvw32PvUHM3GBuSddwdKTg&dib_tag=se&qid=1752072952&s=electronics&sr=1-1091&srs=4551203031'

In [208]:
df_complément[df_complément['High-Tech'] <200].sort_values(by='High-Tech')#.at[4, 'url']

,url,titre,categorie,classement,date_mise_en_ligne,High-Tech,Casques_et_écouteurs,Hygiène_et_Santé,Tensiomètres,Écouteurs_à_conduction_osseuse,...,Housses et étuis pour GPS,Auto et Moto,Bracelets de montres pour femme,Univers Hi-Fi,Cassettes audio,Microphones pour radios,Cartes graphiques,Bracelets énergétiques,Accessoires GPS,Réhausseurs enceintes
2348,https://www.amazon.fr/DJI-Essentiel-%C3%89tanc...,"DJI Bundle Essentiel Osmo Action 5 Pro, Caméra...","[High-Tech, Photo et caméscopes, Appareils pho...","[(164, High-Tech), (1, Caméras embarquées), (1...",24 février 2025,164,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
#-------------merging des df------------------

df_final = pd.merge(df_complément, df_full, how='left', on='url')
df_final= df_final.drop(columns='title', axis=1)
df_final= df_final.dropna(axis=1, how='all')
df_final['classement'] = df_final['classement'].astype(str)
df_final.to_parquet("df_extract.parquet", index=False)

In [19]:
pd.set_option('display.max_columns', None)
df_final.columns

NameError: name 'df_final' is not defined

In [219]:
df_final[df_final['Univers Hi-Fi'].notna()]['url']

3347    https://www.amazon.fr/Linsoul-KZ-AS10-intra-au...
Name: url, dtype: object

In [120]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(df_final, x='Casques et écouteurs')

TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [25]:
def sql(query):
    con = duckdb.connect()
    res = con.execute(query).df()
    con.close()
    return res

class_jour = sql(""" SELECT * from read_parquet('../notebooks/df_extract.parquet')""")

In [26]:
sql("""SELECT * from class_jour""")

,url,titre,categorie,classement,date_mise_en_ligne,High-Tech,Tensiomètres,Mode,Claviers,Bricolage,Enceintes,Commerce,Radio-réveils,PlayStation,Webcams,Impédancemètres,Souris,Informatique,Multiprises,Receveurs,Babyphones,Sèche-cheveux,Poupées,Casques,Fixations,Animalerie,PC,Interphones,Autoradios,TV,Électrocardiographes,Sacs,Sécurité,Trépieds,Monopods,Alcootests,Égalisateurs,Casques et écouteurs,Hygiène et Santé,Écouteurs à conduction osseuse,Montres de poche et goussets pour homme,Enceintes Bluetooth portables,Écouteurs intra-auriculaires,Casques de réalité virtuelle pour téléphone portable,Caméras embarquées,Jeux vidéo,Accessoires de réalité virtuelle autonome,Localisateurs d'objets,Sports et Loisirs,Trackers d'activité,Caméras de corps,GPS de running,Enceintes PC,Enceintes portables et stations d'accueil,Montres connectées,Batteries pour lecteurs MP,Climate Pledge Friendly,Chargeurs secteur pour téléphones portables,Housses de protection,Pèse-personne numériques,Montres connectées tendance pour homme,Tapis de souris,Sirènes de sécurité,Instruments de musique et Sono,Casques pour home studio,Étuis et housses pour casques et écouteurs,Lecteurs de carte mémoire externes,Bateaux et navigation,Étuis et protection de réalité virtuelle pour PC,Pédales pour vélo,Casques et écouteurs pour radios,Caméras étanches,Oreillettes et coussinets pour casques et écouteurs,Micro Casques PC,Instruments et outils chirurgicaux,Enceintes d'extérieur,Alimentation pour ordinateur portable,Enceintes satellites,Raccords de tuyauterie,Lunettes de protection contre la lumière bleue,Transmetteurs FM,Supports pour ordinateur portable,Radio portables,Supports accessoires de sport,Cuisine Camping et randonnée,Accessoires pour lecteurs MP,Casques Bluetooth mono écouteur,Étuis et housses pour lecteur MP,Coques et housses standards pour téléphones portables,GPS et accessoires,Jeux et Jouets,Jouets d'espionnage,Drones radiocommandés,Véhicules contrôlés à distance et par application,Casques et écouteurs pour téléphones fixes,Luminaires et Éclairage,Rubans à LED,Lampes de table,Packs d'enceintes,Enceintes de remplacement,Radios AM,Accessoires de réalité virtuelle pour PC,Bracelets de remplacement pour montres connectées,Batteries de remplacement pour téléphone portable,Hauts parleurs coaxiaux,Enceintes bibliotheques,Tablettes tactiles,Caméras statiques,Systèmes sécurité pour la maison,Prises connectées et intelligentes,Casques pour PlayStation,Accessoires pour haut-parleurs scène,Lecteurs MP,Interrupteurs industriels,Haut-parleurs pour plafond et encastrés,Casques gaming pour PlayStation,Piles alcaline,Protecteurs et films d'écran pour montres connectées,Radio Internet,Appareils respiratoires réutilisables,Brosses à dents rotatives,Casques audio de réalité virtuelle autonome,Masques de sommeil,Extentions d'alimentation,Ceintures pectorales,Souris gaming pour PC,Hubs USB,Montres connectées tendance pour garçon,Systèmes de retours In-Ear,Talkie walkie,Barres de son,Accessoires pour enceintes,Bébé et Puériculture,Beauté et Parfum,Jeux et jouets électroniques,Boomers et subwoofers,Lumières vidéo sur caméra,Casques pour Xbox Series X et S,Disques de nettoyage,Enceintes portables avec prise jack,Connecteurs pour enceintes,Décors pour studio photo,Adaptateurs pour casques et écouteurs,Télémètres laser,Systèmes audio multi-room,Adaptateurs sériel,Chargeurs pour téléphones portables,Casques modulables de moto,Solutions streaming,Détecteurs de fumée,Accessoires téléphones portables,Kits d'accessoires automobiles pour téléphone portable,Caméras espion,Supports enceintes,Interrupteurs d'éclairage mural électrique,Chambres à air pour vélo,Systèmes portables Karaoké,Hélicoptères radiocommandés,Accessoires pour flashs,Étuis pour PlayStation,Accessoires trépieds et monopods,Enceintes et stations d'accueil,Enceintes colonnes,Accessoires IT,Bracelet de montres,Batteries pour GPS,Caméras dômes,Télécommandes audio et vidéo domestiques,Câbles d'alimentation,Transformateurs électrique

In [68]:
#----- tri classement ------

import pandas as pd


hist_class = pd.DataFrame({})

for i in class_jour.index : 
    row = class_jour[['titre','url', 'classement']].loc[[i]]
    if row.notna().all(axis= 1).iloc[0] :
        hist_class = pd.concat([row, hist_class])
    else : 
        continue
    

hist_class

,titre,url,classement
4557,KOAIEZ Écouteurs De Sport À Oreille Ouverte av...,https://www.amazon.fr/KOAIEZ-%C3%89couteurs-Co...,[]
4556,"Diyeeni Collier de Caméra pour Chat, Caméra po...",https://www.amazon.fr/Diyeeni-Objectif-Corpore...,[]
4438,LONLI Hue - pour étui iPhone 16 Plus - étui de...,https://www.amazon.fr/LONLI-Hue-t%C3%A9l%C3%A9...,[]
4395,"Maglens-Caméra de Sport et D'activité, Visusca...",https://www.amazon.fr/Maglens-Cam%C3%A9ra-Spor...,[]
4323,Moondrop Golden Ages TWS Écouteurs 13 mm Full ...,https://www.amazon.fr/Moondrop-Golden-Ages-Mul...,[]
...,...,...,...
4,Disjoncteur WIFI intelligent Interrupteur Conn...,https://www.amazon.fr/Interrupteur-Commutateur...,[]
3,Yinyoo KZ EDX Pro Écouteurs filaires avec Micr...,https://www.amazon.fr/%C3%89couteurs-EDX-Pro-a...,"[(313844, 'High-Tech'), (6886, 'Casques et éco..."
2,Xmenha Écouteurs Bluetooth Écran Touché Contrô...,https://www.amazon.fr/Xmenha-%C3%89couteurs-Bl...,"[(537836, 'High-Tech'), (2869, 'Écouteurs à co..."
1,OMRON RS3 Intelli IT tensiomètre automatique a...,https://www.amazon.fr/Tensiom%C3%A8tre-OMRON-C...,"[(6008, 'Hygiène et Santé'), (33, 'Tensiomètre..."


In [ ]:
classement = row['classement'].iloc[0]

if (
    row.notna().all(axis=1).iloc[0]
    and isinstance(classement, list)
    and len(classement) > 0
):
    hist_class = pd.concat([row, hist_class])



In [58]:
hist_class

,titre,classement,url
4557,KOAIEZ Écouteurs De Sport À Oreille Ouverte av...,[],https://www.amazon.fr/KOAIEZ-%C3%89couteurs-Co...
4556,"Diyeeni Collier de Caméra pour Chat, Caméra po...",[],https://www.amazon.fr/Diyeeni-Objectif-Corpore...
4438,LONLI Hue - pour étui iPhone 16 Plus - étui de...,[],https://www.amazon.fr/LONLI-Hue-t%C3%A9l%C3%A9...
4395,"Maglens-Caméra de Sport et D'activité, Visusca...",[],https://www.amazon.fr/Maglens-Cam%C3%A9ra-Spor...
4323,Moondrop Golden Ages TWS Écouteurs 13 mm Full ...,[],https://www.amazon.fr/Moondrop-Golden-Ages-Mul...
...,...,...,...
4,Disjoncteur WIFI intelligent Interrupteur Conn...,[],https://www.amazon.fr/Interrupteur-Commutateur...
3,Yinyoo KZ EDX Pro Écouteurs filaires avec Micr...,"[(313844, 'High-Tech'), (6886, 'Casques et éco...",https://www.amazon.fr/%C3%89couteurs-EDX-Pro-a...
2,Xmenha Écouteurs Bluetooth Écran Touché Contrô...,"[(537836, 'High-Tech'), (2869, 'Écouteurs à co...",https://www.amazon.fr/Xmenha-%C3%89couteurs-Bl...
1,OMRON RS3 Intelli IT tensiomètre automatique a...,"[(6008, 'Hygiène et Santé'), (33, 'Tensiomètre...",https://www.amazon.fr/Tensiom%C3%A8tre-OMRON-C...


In [14]:
# Trouver l’index de la ligne non vide
idx = class_jour['Réhausseurs enceintes'].first_valid_index()

# Afficher toute la ligne correspondante
print(class_jour.loc[idx])

url                        https://www.amazon.fr/certylu-Haut-Parleur-Ref...
titre                      certylu Haut-Parleur Tube Tube Bass Reflex Ven...
categorie                  [High-Tech, Univers Hi-Fi, Accessoires et cons...
classement                 [(2, 'Réhausseurs enceintes'), (769, "Enceinte...
date_mise_en_ligne                                           15 janvier 2019
                                                 ...                        
Microphones pour radios                                                  NaN
Cartes graphiques                                                        NaN
Bracelets énergétiques                                                   NaN
Accessoires GPS                                                          NaN
Réhausseurs enceintes                                                    2.0
Name: 3602, Length: 241, dtype: object
